In [45]:
# Bibliotecas
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import plotly.graph_objects as go
import plotly.express as px

from warnings import filterwarnings
filterwarnings('ignore')

seed = np.random.seed(1)

# Dados 
matches = pd.read_csv('matches.csv')
worldcups = pd.read_csv('worldcups.csv')

# *Preview de todas as copas do mundo*

In [46]:
from turtle import title

fig = go.Figure()

fig.add_traces(go.Line(x=worldcups['year'], y = worldcups['goals_scored']))
fig.update_layout(title = 'Quantidade de gols ao decorrer das copas')
fig.show()

# *Tratando os dados para a produção*

In [47]:
# Verificando se tem valores nulos
matches.info()
# Valores nulos em: win_conditions; winning_team; losing_team

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            900 non-null    int64 
 1   country         900 non-null    object
 2   city            900 non-null    object
 3   stage           900 non-null    object
 4   home_team       900 non-null    object
 5   away_team       900 non-null    object
 6   home_score      900 non-null    int64 
 7   away_score      900 non-null    int64 
 8   outcome         900 non-null    object
 9   win_conditions  62 non-null     object
 10  winning_team    731 non-null    object
 11  losing_team     731 non-null    object
 12  date            900 non-null    object
 13  month           900 non-null    object
 14  dayofweek       900 non-null    object
dtypes: int64(3), object(12)
memory usage: 105.6+ KB


In [48]:
# Separando variáveis que, talvez, não são necessárias
sem_necessidade = matches[['date','month']]
matches = matches.drop(sem_necessidade, axis=1)

# Tratandos os dados nulos
matches.fillna(0, inplace= True)

In [49]:
# y > 1 = vitoria do home_team
# y == 0 = empate
# y < 1 = vitoria do away_team

valores = []

for i in range(len(matches)):
     valores.append(matches['home_score'][i] - matches['away_score'][i])
     
matches['balanco'] = valores

In [50]:
v = matches[['year','home_team','away_team','home_score','away_score','winning_team','losing_team','balanco']]

# *Analisando as médias e os std gerais da copa*

In [51]:
selecoes_mais_vencedoras = pd.DataFrame(v['winning_team'].value_counts()).head(20)
selecoes_mais_vencedoras = selecoes_mais_vencedoras[1:]
selecoes_mais_vencedoras.reset_index().rename(columns={'index':'pais'},inplace=True)
# Brasil é o pais que mais venceu partidas. Mas porque ele não ganhou todas as copas do mundo?

In [52]:
# Verificando as médias
media = v.groupby(by = 'home_team').mean('balanco').dropna().reset_index()
media = media[['home_team','balanco']].sort_values(by='balanco', ascending = False)

In [53]:
# gráfico da média
px.bar(y=media['home_team'],x=media['balanco'])
# média de balanco de gols/partida da turkia é maior que a do brasil

In [54]:
# Verificando as médias
desvio = v.groupby(by = 'home_team').std().dropna().reset_index()
desvio = desvio[['home_team','balanco']].sort_values(by='balanco', ascending = False)

In [55]:
# gráfico da média
px.bar(y=desvio['home_team'],x=desvio['balanco'])
# as melhores seleçoes são aquelas que são mais consistentes.
# P: O std deve esta entre 1 e 2?

In [56]:
media_desvio = pd.merge(media,desvio,on ='home_team',how='left').dropna()
media_desvio = media_desvio.rename(columns={'balanco_y':'desvio','balanco_x':'media'})
media_desvio.reset_index().drop('index', axis=1, inplace=True)

In [57]:
# Ordenando a tabela de cima baseado nas seleçoes que mais venceram copa (n é a partida)
paises_com_titulos = worldcups.sort_values(by = 'year', ascending=False).value_counts(subset='winner').reset_index().rename(columns={0:'titulos'})

In [ ]:
paises_para_analise = []
for i in selecoes_mais_vencedoras.index:
    paises_para_analise.append(i)

stats = {'pais':[],'media':[],'desvio':[]}

for i in range(len(paises_para_analise)):
    a = paises_para_analise[i]
    aa = (media_desvio.loc[(media_desvio['home_team']== a)])
    recorte = str(aa).split()
    stats['pais'].append(recorte[-3])
    stats['media'].append(recorte[-2])
    stats['desvio'].append(recorte[-1])

stats = pd.DataFrame(stats).dropna()
# Nessa tabela temos as 20 selecoes que mais ganharam partidas

# *Analisando as médias e std das seleções em cada copa*

In [ ]:
# A gente pode analisando em cada copa a média e os std das seleções... 
# Acredito que será melhor tanto para analisar quanto para a predição.


In [ ]:
# Segundo Mario Filho, encontrar o valor dos gols de cada time em cada partida podemos econtrar a melhor performace. A variavel dependente será essa.

# Separar os dados através dos anos. Ou seja, vou escolher e treinar o modelo com as 14 primeiras copas (70% dos dados) 
# e validar com as 6 ultimas copas (30% dos dados)